Script to calculate return periods of the observed water level data. Can either use the complete time series or the 2003-2023 period (i.e., GloFAS analysis period). 

In [1]:
# import relevant packages
import pandas as pd
import numpy as np
from scipy.stats import genextreme
import os
from pathlib import Path

In [2]:
# define country and directory
country = 'mozambique'
#directory = Path(f'/s3/scratch/jamie.towner/flood_aa/{country}')
directory = Path(r"C:\Users\15133\Documents\WFP\flood_hazard\flood_aa\MOZ_training")  # define main working directory
output_directory = directory / "outputs/thresholds"
Path(output_directory).mkdir(parents=True, exist_ok=True)  # create directory if it does not already exist 

In [3]:
# define paths to data
observed_data_directory = directory / "data/observations/gauging_stations/all_stations"
observed_data_file = "observations_complete_series.csv" # use observations.csv for 2003-2023 period

# load data
observed_data_path = directory / observed_data_directory /observed_data_file
observed_data = pd.read_csv(observed_data_path)

In [4]:
# convert date columns to datetime
observed_data["date"] = pd.to_datetime(observed_data["date"], format='mixed')

In [5]:
# check data
observed_data.head()

,date,Limpopo_em_Mapai,Limpopo__Combomune,Limpopo_em_Chokwe,Limpopo_em_Sicacate,Limpopo_em_Mabalane,Limpopo_em_XaiXai,Changane_em_VGomes_da_Costa,Changane_em_Chibuto,Limpopo_em_Macaretane,Elefantes_em_Munhamane,Singuezi_em_Bingo
0,1951-05-01,NaN,NaN,NaN,NaN,NaN,0.946667,NaN,NaN,NaN,NaN,NaN
1,1951-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1951-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1951-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1951-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# function to calculate return periods for a given station's data
def calculate_return_periods(station_data, years=[2, 5, 10, 20]):
    # drop NA values
    station_data = station_data.dropna().copy()

    # extract the year from the date column 
    station_data.loc[:, 'year'] = station_data['date'].dt.year
    
    # group by year and get the maximum value for each year 
    annual_max = station_data.groupby('year')[station_data.columns[1]].max()
    
    # fit the data to a GEV distribution (Generalized Extreme Value distribution)
    #params = genextreme.fit(annual_max)

    # calculate the return period for each year 
    return_periods = {}
    for return_year in years:
        # the formula for return period is: 1 / (1 - F(x))
        # F(x) is the CDF of the fitted distribution at the threshold (max value)
        threshold = np.percentile(annual_max, 100 * (1 - 1/return_year))
        #threshold = genextreme.ppf(1 - 1/return_year, *params)
        return_periods[return_year] = threshold

    return return_periods

# initialize a dictionary to store return periods for each station
return_periods_dict = {}

# iterate over each station in the observed_data 
for station in observed_data.columns:
    if station == 'date':
        continue  # Skip 'date' column

    # get the data for this station
    station_data = observed_data[['date', station]]

    # skip if all values are NaN
    if station_data[station].dropna().empty:
        continue

    # calculate return periods for the station
    return_periods = calculate_return_periods(station_data)
    
    # store the return periods in the dictionary
    return_periods_dict[station] = return_periods

# convert the dictionary to a dataframe
return_periods_df = pd.DataFrame.from_dict(return_periods_dict, orient='index')

In [7]:
# check the output
return_periods_df

,2,5,10,20
Limpopo_em_Mapai,4.365000,5.550667,6.377333,7.129333
Limpopo__Combomune,5.313333,7.452000,8.418000,9.973667
Limpopo_em_Chokwe,4.863333,7.110000,7.674667,8.037333
Limpopo_em_Sicacate,7.563333,10.174000,10.632000,11.019000
Limpopo_em_Mabalane,5.483333,7.938667,8.404333,9.150500
Limpopo_em_XaiXai,2.930000,4.155000,5.178000,5.505000
Changane_em_VGomes_da_Costa,1.410000,3.824000,5.547000,5.760500
Changane_em_Chibuto,1.495000,3.543333,6.762500,6.888917
Limpopo_em_Macaretane,97.543333,98.665333,99.039000,99.657667
Elefantes_em_Munhamane,5.000000,7.368667,8.066000,8.423000


In [8]:
# save output as a csv 
return_periods_df.to_csv(os.path.join(output_directory, "observed_return_periods_complete_series.csv"), index=True)